# Convolutional Neural Network for image classification

## Genesis Quiles-Galarza

We will train convolutional neural networks to solve a binary classification task on a realistic case involving images obtained from image flow cytometry of red blood cells (available for download in the Github repository). The total number of provided images is 3469 and the original image resolution is $200×200$. We will use 80% of the data to train your model and 20% to validate your predictions. We will experiment with different convolutional architectures, pre-processing or data augmentation techniques (e.g. image downsampling, translations, rotations, etc.).


In [25]:
import time
import timeit
import numpy as np
import matplotlib.pyplot as plt
from numpy import *
import pylab as p
import scipy
from scipy import integrate
from pyDOE import *
import torch
import torch.nn as nn
import torch.utils.data
import torch.nn.functional as F
from torch.autograd import Variable
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from sklearn.model_selection import train_test_split
import cv2
import os
from PIL import Image

In [101]:
# This code is originally sourced and has been modified from Paris Perdikaris's
# Data Loader code posted on Github and found at the following link:
# https://github.com/PredictiveIntelligenceLab/ENM531/blob/master/Week7/dataloader.py

# Data Loader
np.random.seed(1234)
torch.manual_seed(1234)

class Cell_dataset(torch.utils.data.Dataset):
    """Pocked Cell dataset."""
    def __init__(self, file_location):
        """
        Args:
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.location = file_location
        self.filenames = os.listdir(self.location)
        # self.transform = transform

    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        name = self.filenames[idx]
        #img = cv2.imread(self.location+name, cv2.IMREAD_GRAYSCALE)
        #img = np.array(img)
        #img = img.astype('float')
        #img = np.reshape(img,(1,200,200))
        nm = name.split("_")
        if nm[0]=="Pocked":
            label = 1
        elif nm[0]=="Unpocked":
            label = 0
        else:
            print("String Parsing Error")
        
        label = np.array(label).astype('float')
        label = np.reshape(label,(1))
        #Apply transformations here
        img = Image.open(self.location + name)
        img = transforms.functional.to_grayscale(img)
        img = transforms.functional.resize(img, size=(20, 20))
        h_flip = transforms.RandomHorizontalFlip(0.25)
        v_flip = transforms.RandomVerticalFlip(0.25)
        rotate = transforms.RandomRotation((-180, 180))
        img = h_flip(img)
        img = v_flip(img)
        img = rotate(img)
        img_to_tensor = transforms.ToTensor()
        img = img_to_tensor(img)
        return img, torch.from_numpy(label).type(torch.LongTensor)

In [97]:
# This code is originally sourced and has been modified from Paris Perdikaris's
# CNN code posted on Github and found at the following link:
# https://github.com/PredictiveIntelligenceLab/ENM531/blob/master/Week7/models_torch.py

# Define CNN architecture and forward pass
class CNN(torch.nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = torch.nn.Conv2d(1, 16, kernel_size=3, padding=1)
        self.norm1 = torch.nn.BatchNorm2d(16)
        self.conv2 = torch.nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.norm2 = torch.nn.BatchNorm2d(32)
        self.conv3 = torch.nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.norm3 = torch.nn.BatchNorm2d(64)
        self.pool = torch.nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc = torch.nn.Linear(64 * 2 * 2, 2)
        self.dropout = torch.nn.Dropout2d(0.1)
        self.ReLU = torch.nn.ReLU()
        
    def forward_pass(self, x):
        out = self.ReLU(self.norm1(self.conv1(x)))
        out = self.pool(out)
        # out = self.dropout(out)
        out = self.ReLU(self.norm2(self.conv2(out)))
        out = self.pool(out)
        # out = self.dropout(out)
        out = self.ReLU(self.norm3(self.conv3(out)))
        out = self.pool(out)
        out = out.view(out.size(0), -1)
        # out = self.dropout(out)
        out = self.fc(out)
        return out


class ConvNet:
    # Initialize the class
    def __init__(self, train_data, test_data):  
        self.train_data = train_data
        self.test_data = test_data
        self.net = CNN()
        self.loss_fn = torch.nn.CrossEntropyLoss()
        self.optimizer = torch.optim.Adam(self.net.parameters(), lr=1e-3)
        
           
    # Trains the model by minimizing the Cross Entropy loss
    def train(self, num_epochs = 10, batch_size = 128):
        # Create a PyTorch data loader object
        self.trainloader = torch.utils.data.DataLoader(self.train_data, batch_size=batch_size, shuffle=True)
        for epoch in range(num_epochs):
            for it, (images, labels) in enumerate(self.trainloader):
                images = torch.autograd.Variable(images)
                labels = torch.autograd.Variable(labels)
                self.optimizer.zero_grad()
                outputs = self.net.forward_pass(images)
                labels = labels.view(labels.shape[0])
                loss = self.loss_fn(outputs, labels)
                loss.backward()
                self.optimizer.step()
                if it % 10 == 0:
                    print ("Epoch: {} | Iter: {} | Loss: {}".format(epoch+1, it, loss.data)) 
                   
                    
    def test(self, batch_size = 128):
        test_loader = torch.utils.data.DataLoader(self.test_data, batch_size=batch_size, shuffle=True)
        # Test prediction accuracy
        correct = 0
        total = 0
        for i, (images, labels) in enumerate(test_loader):
            images = torch.autograd.Variable(images)
            outputs = self.net.forward_pass(images)
            _, predicted = torch.max(outputs.cpu().data, 1)
            total += labels.shape[0]
            correct += (predicted == labels).sum().item()
        print("Test accuracy on {} images: {:.4f}%".format(len(self.test_data), correct / total))
        
    
    # Evaluates predictions at test points    
    #def predict(self, X_star):
    #    X_star = torch.from_numpy(X_star).type(self.dtype_double) 
    #    X_star = Variable(X_star, requires_grad=False)
    #    y_star = self.net.forward_pass(X_star)
    #    y_star = y_star.cpu().data.numpy()
    #    return y_star

This run applies no transformations to the images with number of epochs = 10 and batch size = 128.

In [98]:
test_data = Cell_dataset('test/')
train_data = Cell_dataset('train/')

cnn = ConvNet(train_data, test_data)
cnn.train()
cnn.test()

Epoch: 1 | Iter: 0 | Loss: 0.8126410245895386
Epoch: 1 | Iter: 10 | Loss: 0.3877248167991638
Epoch: 1 | Iter: 20 | Loss: 0.47052258253097534
Epoch: 2 | Iter: 0 | Loss: 0.3528001606464386
Epoch: 2 | Iter: 10 | Loss: 0.2877894937992096
Epoch: 2 | Iter: 20 | Loss: 0.27889057993888855
Epoch: 3 | Iter: 0 | Loss: 0.3289453983306885
Epoch: 3 | Iter: 10 | Loss: 0.33050575852394104
Epoch: 3 | Iter: 20 | Loss: 0.34997984766960144
Epoch: 4 | Iter: 0 | Loss: 0.3750849664211273
Epoch: 4 | Iter: 10 | Loss: 0.32393625378608704
Epoch: 4 | Iter: 20 | Loss: 0.3052484691143036
Epoch: 5 | Iter: 0 | Loss: 0.2982499599456787
Epoch: 5 | Iter: 10 | Loss: 0.3614969849586487
Epoch: 5 | Iter: 20 | Loss: 0.33477067947387695
Epoch: 6 | Iter: 0 | Loss: 0.3695635497570038
Epoch: 6 | Iter: 10 | Loss: 0.2905466854572296
Epoch: 6 | Iter: 20 | Loss: 0.2598809003829956
Epoch: 7 | Iter: 0 | Loss: 0.29965856671333313
Epoch: 7 | Iter: 10 | Loss: 0.34567078948020935
Epoch: 7 | Iter: 20 | Loss: 0.32998472452163696
Epoch: 8 | 

This run applies no random transformations to the images with number of epochs = 10 and batch size = 64.

In [99]:
num_epochs = 10 
batch_size = 64

cnn = ConvNet(train_data, test_data)
cnn.train(num_epochs, batch_size)
cnn.test(batch_size)

Epoch: 1 | Iter: 0 | Loss: 0.8099663853645325
Epoch: 1 | Iter: 10 | Loss: 0.4683122932910919
Epoch: 1 | Iter: 20 | Loss: 0.3403487801551819
Epoch: 1 | Iter: 30 | Loss: 0.359496533870697
Epoch: 1 | Iter: 40 | Loss: 0.4109576344490051
Epoch: 2 | Iter: 0 | Loss: 0.2967786490917206
Epoch: 2 | Iter: 10 | Loss: 0.2813061773777008
Epoch: 2 | Iter: 20 | Loss: 0.4074293076992035
Epoch: 2 | Iter: 30 | Loss: 0.4051297903060913
Epoch: 2 | Iter: 40 | Loss: 0.34555450081825256
Epoch: 3 | Iter: 0 | Loss: 0.35366693139076233
Epoch: 3 | Iter: 10 | Loss: 0.3505580425262451
Epoch: 3 | Iter: 20 | Loss: 0.2707765996456146
Epoch: 3 | Iter: 30 | Loss: 0.23460683226585388
Epoch: 3 | Iter: 40 | Loss: 0.37340474128723145
Epoch: 4 | Iter: 0 | Loss: 0.3771206736564636
Epoch: 4 | Iter: 10 | Loss: 0.20156748592853546
Epoch: 4 | Iter: 20 | Loss: 0.37737375497817993
Epoch: 4 | Iter: 30 | Loss: 0.36035579442977905
Epoch: 4 | Iter: 40 | Loss: 0.33306723833084106
Epoch: 5 | Iter: 0 | Loss: 0.43771010637283325
Epoch: 5 |

This run applies no transformations but doubles the amount of epochs.

In [100]:
num_epochs = 20 
batch_size = 128

cnn = ConvNet(train_data, test_data)
cnn.train(num_epochs, batch_size)
cnn.test(batch_size)

Epoch: 1 | Iter: 0 | Loss: 0.9448046684265137
Epoch: 1 | Iter: 10 | Loss: 0.2997686266899109
Epoch: 1 | Iter: 20 | Loss: 0.40549200773239136
Epoch: 2 | Iter: 0 | Loss: 0.34627529978752136
Epoch: 2 | Iter: 10 | Loss: 0.3997986912727356
Epoch: 2 | Iter: 20 | Loss: 0.32744526863098145
Epoch: 3 | Iter: 0 | Loss: 0.2999429702758789
Epoch: 3 | Iter: 10 | Loss: 0.3831724524497986
Epoch: 3 | Iter: 20 | Loss: 0.33958277106285095
Epoch: 4 | Iter: 0 | Loss: 0.3591929078102112
Epoch: 4 | Iter: 10 | Loss: 0.3027552664279938
Epoch: 4 | Iter: 20 | Loss: 0.255239874124527
Epoch: 5 | Iter: 0 | Loss: 0.31683242321014404
Epoch: 5 | Iter: 10 | Loss: 0.28196799755096436
Epoch: 5 | Iter: 20 | Loss: 0.29159659147262573
Epoch: 6 | Iter: 0 | Loss: 0.3352051377296448
Epoch: 6 | Iter: 10 | Loss: 0.2568396031856537
Epoch: 6 | Iter: 20 | Loss: 0.35424166917800903
Epoch: 7 | Iter: 0 | Loss: 0.22989685833454132
Epoch: 7 | Iter: 10 | Loss: 0.30857276916503906
Epoch: 7 | Iter: 20 | Loss: 0.2895723283290863
Epoch: 8 | 

The previous run had the best accuracy, let's try applying some random transformations to the images and seeing how that affects the loss and accuracy.

In [102]:
num_epochs = 20 
batch_size = 128

cnn = ConvNet(train_data, test_data)
cnn.train(num_epochs, batch_size)
cnn.test(batch_size)

Epoch: 1 | Iter: 0 | Loss: 0.8385254144668579
Epoch: 1 | Iter: 10 | Loss: 0.4421956539154053
Epoch: 1 | Iter: 20 | Loss: 0.3397660255432129
Epoch: 2 | Iter: 0 | Loss: 0.36229249835014343
Epoch: 2 | Iter: 10 | Loss: 0.3292085826396942
Epoch: 2 | Iter: 20 | Loss: 0.333326131105423
Epoch: 3 | Iter: 0 | Loss: 0.2863457500934601
Epoch: 3 | Iter: 10 | Loss: 0.3336540758609772
Epoch: 3 | Iter: 20 | Loss: 0.2811071276664734
Epoch: 4 | Iter: 0 | Loss: 0.30463820695877075
Epoch: 4 | Iter: 10 | Loss: 0.26561760902404785
Epoch: 4 | Iter: 20 | Loss: 0.33842235803604126
Epoch: 5 | Iter: 0 | Loss: 0.3095731735229492
Epoch: 5 | Iter: 10 | Loss: 0.3091684579849243
Epoch: 5 | Iter: 20 | Loss: 0.32981017231941223
Epoch: 6 | Iter: 0 | Loss: 0.32305464148521423
Epoch: 6 | Iter: 10 | Loss: 0.377492219209671
Epoch: 6 | Iter: 20 | Loss: 0.2213761955499649
Epoch: 7 | Iter: 0 | Loss: 0.29459109902381897
Epoch: 7 | Iter: 10 | Loss: 0.31197166442871094
Epoch: 7 | Iter: 20 | Loss: 0.2975875735282898
Epoch: 8 | Ite

The transformations did not affect the outcome that much - which makes sense when you look at the images. They are all very similar in appearance and there's not much difference between a rotated or a flipped image.

Unfortunately, I couldn't get this function to have any higher accuracy than ~60%. Some other options for improving this accuracy are downsampling.